In [45]:
import requests
from bs4 import BeautifulSoup
import json
import re
import os

In [44]:
#Webscraping der Schussdaten von dem Spiel Fc Augsburg - Holstein Kiel 
url = 'https://understat.com/match/28025'
variable_name='shotsData'
response= requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
scripts = soup.find_all('script')
for script in scripts:
    if variable_name in script.text:
        ##Json-Rohdaten finden
        start = script.text.find('JSON.parse("') + len('JSON.parse("')
        end = script.text.find('");', start)
        ##Rohdaten so manipulieren, dass nur noch die escape-Daten vorhanden sind
        json_raw = script.text[start:end]
        x= json_raw.find('(')
        y= json_raw.find(')')
        json_raw=json_raw[x+2:y-1]
        ## von escape Rohdaten in normaler JSON-Form umwandlen
        json_clean = json_raw.encode().decode('unicode_escape')
        ##in python-Dictonary umwandeln
        data = json.loads(json_clean)
        ## in Datei abspeichern
        with open("meinedaten.json", "w") as outfile:
            json.dump(data, outfile, indent=4)
        

In [46]:
print(os.getcwd())

C:\Users\Schönherr
